### RAG 구조와 Chroma 벡터 DB를 활용해 질의하고 응답을 비교

In [1]:
from openai import OpenAI
from getpass import getpass
from langchain.document_loaders import PyPDFLoader, TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# LangChain과 연동하여 Chroma DB의 기능을 약식으로 구현할 수 있게 해주는 Chroma 클래스
from langchain.vectorstores import Chroma
# OpenAI 지원 사전 학습된 임베딩 모델 임포트
from langchain.embeddings.openai import OpenAIEmbeddings

In [2]:
MY_API_KEY = getpass("OpenAI API Key >> ")

In [3]:
client = OpenAI(api_key = MY_API_KEY)

In [4]:
completion = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = [{"role" : "user", "content" : "기술 평가가 뭐야?"}],
    temperature = 0
)
print(completion.choices[0].message.content)

기술평가는 특정 기술이나 제품의 성능, 품질, 안전성, 신뢰성 등을 평가하는 과정을 말합니다. 이를 통해 해당 기술이나 제품의 우수성과 개선이 필요한 부분을 파악할 수 있습니다. 주로 실험, 시험, 분석 등의 방법을 사용하여 기술적인 측면을 평가하게 됩니다.


### 1. RAG 구조와 Chroma를 활용해 질의

In [5]:
loaders = [PyPDFLoader("../data/기술보증기금과 한국경제.pdf"), PyPDFLoader("../data/기술보증기금 주요업무.pdf")]

pdf_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50,
    separators = ["\n\n", "\n", " "]
)

In [6]:
my_chunks = []
for i in loaders :
    # 2개의 파일들로부터 추출 된 각각의 document 객체들을 하나의 리스트에 병합
    my_chunks.extend(i.load_and_split(text_splitter = pdf_splitter))
    
print(len(my_chunks))
my_chunks

14


[Document(page_content='페이지  1 / 5 \n \n기술보증기금과  한국경제  \n \nI. 기술보증기금  개요  \n1. 설립근거  : 기술보증기금법  \n \n- 설립목적 (존재이유 ) \n✓ 기술보증기금을  설립하여  기술보증제도를  정착·발전시킴으로써  신기술사업에  대한 자금의  \n공급을  원활하게  하고 나아가  국민경제의  발전에  이바지함을  목적으로  함(기술보증기금법 1\n조) \n✓ 설립 : 담보능력이  미약한  기업의  채무를  보증하게  하여 기업에 대한 자금 융통을  원활하게  하기 \n위하여  기술보증기금을  설립(법 12조) \n✓ 기금의  재원 : 정부 출연금의  예산은  중소벤처기업부  소관으로  함 \n \n☞ 기술보증기금은   \n✓ 기술력은  우수하지만  담보 부족한  중소기업의  \n✓ 기술성과  사업성  평가를  통해 기술보증을  지원하며 , \n✓ 기술평가 , 벤처이노비즈기업  인증, 중소기업  창업지원  등의 업무 수행 \n \n2. 주요개념1 \n \n업  무 내  용 \n기술보증  신기술사업자가  부담하는  금전 채무 보증.(신용부족 -담보부족  해결) \n \n금융회사  등으로부터  자금 대출을 받음으로써  금융회사  등에 대하여  부담하는  금전 채무를 \n기술보증기금이  기술보증서로  보증 \n 신기술사업자  - 기술을  개발하거나  이를 응용하여  사업화하는  중소기업 (「중소기업기본법」에  \n따른 중소기업 ) 및 대통령령으로  정하는  기업 \n- "기업"이란 사업을  하는 개인 및 법인 \n신용보증  상시 사용하는  종업원이  1천명 이하이고  총자산액이  1천억원  이하인  기업이  부담하는  금전\n채무 보증. \n기술평가  해당 기술과  관련된  기술성·시장성·사업 타당성 등을 종합적으로  평가하여  \n금액·등급·의견  또는 점수 등으로  표시하는  것을 말한다  \n* 기금은  기술평가의  객관성  및 공정성  등을 확보하기  위하여  기술평가의  \n기준·절차·방법·종

### 2. 임베딩 설정 및 Chroma DB 생성

In [7]:
# OpenAI에서 지원해 주는 사전 학습된 임베딩 모델을 적용한 객체 생성
my_embedding = OpenAIEmbeddings(
    model = "text-embedding-ada-002",
    api_key = MY_API_KEY
)

C:\Dev\tools\Anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


- Chroma DB 생성

In [8]:
# DB 경로 설정
my_directory = "../data/VectorStores/"

# from_documents() : 청킹된 문서(documents 객체)의 정보 및 임베딩, 경로 정보로 Chroma DB 생성
vectordb = Chroma.from_documents(
    documents = my_chunks,
    embedding = my_embedding,
    persist_directory = my_directory
)

#### 한 번 생성된 DB를 불러와서 사용
- 같은 경로에 DB를 계속 생성하면 중복된 데이터가 계속 누적되어 저장 됨
- 한 번 생성된 DB를 불러와서 사용하는 방식

In [10]:
my_embedding = OpenAIEmbeddings(
    model = "text-embedding-ada-002",
    api_key = MY_API_KEY
)
my_directory = "../data/VectorStores/"

# 불러올 때는 embedding_function 매개 변수에 임베딩 정보 넣기
vectordb = Chroma(
    embedding_function = my_embedding,
    persist_directory = my_directory
)

#### Chroma DB 활용
1) DB 정보 확인

In [11]:
# 저장되어 있는 청크 갯수 확인
vectordb._collection.count()

14

In [12]:
# 저장되어 있는 청크에 대한 세부 정보 확인
vectordb._collection.get()

{'ids': ['289ec60c-fd06-4e9d-a9b3-b58067e38271',
  '2a69124c-92cb-4b35-a2d5-a0a7c39e34fd',
  '3ff1eb26-b8a5-4207-abd1-73abffad0387',
  '496b001e-fd4d-4952-992f-647b0f421f36',
  '54ea3ee5-843e-41f6-9bff-a1f39e89f24a',
  '56684bd9-9928-4030-b22f-b66ab599f510',
  '58624b69-df27-4062-bc3d-a3f09b3d6542',
  '94e59b18-3c37-4eb0-86db-d622977eeaa2',
  '96f85770-cb02-455b-9ea6-d69205c798eb',
  'bd5a31dc-3354-40b6-afaf-cc0ebe9da8c1',
  'ddbc3aae-e6ff-48b3-b921-5d1d9e1071ec',
  'f3c69e3b-c232-4864-8f23-de31507b06ae',
  'f5ea7753-4b72-465a-858e-5e75d61e8ce5',
  'ff1c14b3-5cec-4178-93c7-ad536488cced'],
 'embeddings': None,
 'metadatas': [{'page': 0, 'source': '../data/기술보증기금 주요업무.pdf'},
  {'page': 1, 'source': '../data/기술보증기금 주요업무.pdf'},
  {'page': 2, 'source': '../data/기술보증기금 주요업무.pdf'},
  {'page': 0, 'source': '../data/기술보증기금 주요업무.pdf'},
  {'page': 3, 'source': '../data/기술보증기금과 한국경제.pdf'},
  {'page': 0, 'source': '../data/기술보증기금과 한국경제.pdf'},
  {'page': 3, 'source': '../data/기술보증기금과 한국경제.pdf'},
  {

#### 2) DB 값 추가
- DB에 텍스트 추가하기

In [13]:
vectordb.add_texts(texts = ["====WOODZ====", "====Sample Text===="])

['8c42b33e-316b-41dd-87d8-7c99799eb962',
 'd70b7dbb-023a-49bd-9224-4e8b21bd7d5f']

In [14]:
vectordb._collection.get()

{'ids': ['289ec60c-fd06-4e9d-a9b3-b58067e38271',
  '2a69124c-92cb-4b35-a2d5-a0a7c39e34fd',
  '3ff1eb26-b8a5-4207-abd1-73abffad0387',
  '496b001e-fd4d-4952-992f-647b0f421f36',
  '54ea3ee5-843e-41f6-9bff-a1f39e89f24a',
  '56684bd9-9928-4030-b22f-b66ab599f510',
  '58624b69-df27-4062-bc3d-a3f09b3d6542',
  '8c42b33e-316b-41dd-87d8-7c99799eb962',
  '94e59b18-3c37-4eb0-86db-d622977eeaa2',
  '96f85770-cb02-455b-9ea6-d69205c798eb',
  'bd5a31dc-3354-40b6-afaf-cc0ebe9da8c1',
  'd70b7dbb-023a-49bd-9224-4e8b21bd7d5f',
  'ddbc3aae-e6ff-48b3-b921-5d1d9e1071ec',
  'f3c69e3b-c232-4864-8f23-de31507b06ae',
  'f5ea7753-4b72-465a-858e-5e75d61e8ce5',
  'ff1c14b3-5cec-4178-93c7-ad536488cced'],
 'embeddings': None,
 'metadatas': [{'page': 0, 'source': '../data/기술보증기금 주요업무.pdf'},
  {'page': 1, 'source': '../data/기술보증기금 주요업무.pdf'},
  {'page': 2, 'source': '../data/기술보증기금 주요업무.pdf'},
  {'page': 0, 'source': '../data/기술보증기금 주요업무.pdf'},
  {'page': 3, 'source': '../data/기술보증기금과 한국경제.pdf'},
  {'page': 0, 'source': '

- DB에서 문서 추가하기

In [15]:
add_loaders = [PyPDFLoader("../data/기술보증기금 주요업무_기술보증.pdf"), PyPDFLoader("../data/기술보증기금 주요업무_기술평가.pdf")]
add_chunks = []

for i in add_loaders :
    add_chunks.extend(i.load_and_split(text_splitter = pdf_splitter))

print(len(add_chunks))

# 문서 추가 명령 실행
vectordb.add_documents(documents = add_chunks)

17


['db7aea6a-7f4f-4e26-bf6b-42d29444f8fe',
 '63e77fe0-05ee-435b-989b-838110e7799c',
 '322aad00-7d5d-4839-baae-4f37bbdd400c',
 'e05329a7-0f3d-4366-90fb-6809bd0c4565',
 '53a79f52-e137-47b6-af34-efdc49cd46f6',
 '12d093bf-9eef-4f4c-b9f6-6d3248281643',
 '79c29807-88c7-4164-ac49-9e85a7c277a4',
 '74cd200b-98f9-4d0e-acb7-33c33bd304c9',
 '28a302c9-340f-4f62-b6bc-fee9df171925',
 'b20b53bb-f5c8-4bde-936a-3f0ec432a81e',
 'dbe4fe63-fb3e-470a-9e5a-dad8ff2460c8',
 'aebaacfe-fd79-4cac-b14b-017809b1de87',
 'c5ec44fc-6f6c-4c1a-80af-553b9fceee12',
 '5de870ce-1e93-44f2-a5f0-459161aaf464',
 '6657a1e9-ae56-4da8-a942-9706375105f4',
 'd9bdd2d9-b7e0-49c5-a07c-9f1e38f22d15',
 '15d79bc2-57e1-4195-a45d-a9042ecefce8']

In [16]:
vectordb._collection.count()

33

#### 3) DB 값 삭제

In [17]:
# id 값으로 삭제
vectordb.delete(ids = ["8c42b33e-316b-41dd-87d8-7c99799eb962", "d70b7dbb-023a-49bd-9224-4e8b21bd7d5f"])
vectordb._collection.count()

31

In [ ]:
# 모든 데이터 삭제
vectordb.delete_collection()

In [ ]:
vectordb._collection.count()
# 저장 된 값이 없이 실행 시, 오류 발생

#### 4) 유사도 검색
- 벡터DB에서 지원하는 간단한 유사도 검색

In [18]:
question = "기술 평가가 뭐야?"

# similarity_search() : 유사도 검색 실행 (k = 반환 받고자 하는 청크 갯수, default는 4)
similar_docs = vectordb.similarity_search(question, k = 3)

print(len(similar_docs))
print(similar_docs)

3
[Document(page_content='페이지  2 / 6 \n \n (2) 기술평가3 \n기술평가란 ?- 대상기술의  기술성 , 시장성 , 사업타당성  등을 분석하고  결과를  금액, 등급, 의견 등으로  \n표현하는  것이다 .  \n✓ 기술평가는  기술수준  등 기술자체에  대한 평가를  표현한  용어지만 , 기술과  기업(사업)간의 밀접한  관련\n성으로  인해서  최근에는  위의 의미로  사용됨  \n✓ 기술가치평가는  기술평가 +가치평가가  결합된  용어로서 , 기술적인  요소를  기반으로  시장에서의  가치를  \n평가 하는 것을 의미 \n \n (3) 기술평가  유형 \n 정 의 세부평가종류  \n기술가치평\n가 대상 기술에  의하여  현재 시현되고  있거나  \n장래에  시현될  기술의  가치를  평가하여  평\n가결과를  금액으로  표시 - 벤처기업  현물출자  특례대상  산업재산권   등의 평가 \n- 기술의  담보가치를  산정하기  위한 평가 \n- 기술이전ㆍ거래4 기준가격  산정을  위한   평가 \n- 기술사업의  이전ㆍ양수도를  위한 평가 등  \n기술사업  \n타당성평가  기업이  특정기술 (OR 아이디어)을 사업화 , \n추진중인  기술사업의  투자 확대 시 사업의  \n기술성  및 사업타당성  평가 - 벤처기업  확인, INNO -BIZ 선정평가  \n- 정책자금  지원 대상자  선정 위한 평가 \n- 금융기관  등의 여신심사용  기술평가  \n- 보증지원을  위한 평가 등  \n종합기술평\n가 기업 보유 기술을  경영환경 , 사업전망  등\n을 종합 평가 - 금융기관 , 벤처캐피탈  또는 엔젤투자자  등 투자용  평가 \n- 벤처기업의  코스닥시장  상장 위한 평가 \n- 주식가치평가  등  \n(4)기술평가의  범위 : 기술성 , 시장성 , 사업성을  종합적으로  검토 \n① 기술성  기술의 수준, 기술의  활용성 , 기술의  파급성  \n② 시장성  시장규모  및 특성, 동업계  현황, 시장수요전망  \n③ 사업성 

In [19]:
similar_docs[0].page_content

'페이지  2 / 6 \n \n (2) 기술평가3 \n기술평가란 ?- 대상기술의  기술성 , 시장성 , 사업타당성  등을 분석하고  결과를  금액, 등급, 의견 등으로  \n표현하는  것이다 .  \n✓ 기술평가는  기술수준  등 기술자체에  대한 평가를  표현한  용어지만 , 기술과  기업(사업)간의 밀접한  관련\n성으로  인해서  최근에는  위의 의미로  사용됨  \n✓ 기술가치평가는  기술평가 +가치평가가  결합된  용어로서 , 기술적인  요소를  기반으로  시장에서의  가치를  \n평가 하는 것을 의미 \n \n (3) 기술평가  유형 \n 정 의 세부평가종류  \n기술가치평\n가 대상 기술에  의하여  현재 시현되고  있거나  \n장래에  시현될  기술의  가치를  평가하여  평\n가결과를  금액으로  표시 - 벤처기업  현물출자  특례대상  산업재산권   등의 평가 \n- 기술의  담보가치를  산정하기  위한 평가 \n- 기술이전ㆍ거래4 기준가격  산정을  위한   평가 \n- 기술사업의  이전ㆍ양수도를  위한 평가 등  \n기술사업  \n타당성평가  기업이  특정기술 (OR 아이디어)을 사업화 , \n추진중인  기술사업의  투자 확대 시 사업의  \n기술성  및 사업타당성  평가 - 벤처기업  확인, INNO -BIZ 선정평가  \n- 정책자금  지원 대상자  선정 위한 평가 \n- 금융기관  등의 여신심사용  기술평가  \n- 보증지원을  위한 평가 등  \n종합기술평\n가 기업 보유 기술을  경영환경 , 사업전망  등\n을 종합 평가 - 금융기관 , 벤처캐피탈  또는 엔젤투자자  등 투자용  평가 \n- 벤처기업의  코스닥시장  상장 위한 평가 \n- 주식가치평가  등  \n(4)기술평가의  범위 : 기술성 , 시장성 , 사업성을  종합적으로  검토 \n① 기술성  기술의 수준, 기술의  활용성 , 기술의  파급성  \n② 시장성  시장규모  및 특성, 동업계  현황, 시장수요전망  \n③ 사업성  경쟁력, 사업추진능력 , 재무구조  등'

In [20]:
# as_retriever() : 정의된 벡터 DB를 정보 검색기로 사용할 수 있게 해주는 함수 (k = 출력 청크 갯수)
my_retriever = vectordb.as_retriever(search_kwargs = {"k" : 3})

# 검색 방법은 기본적으로 유사도 검색 알고리즘을 사용하나 다른 검색 기능으로의 확장도 가능함
# 검색기에 입력된 질의에 대해 관련 있는 청크를 반환 (유사도 검색)
relevant_docs = my_retriever.get_relevant_documents("기술 평가가 뭐야?")
relevant_docs

# RetrievalQA : LangChain에서 지원하는 질의 응답 수행 클래스
# 위에서 만든 검색기(my_retreiver)와 LLM 모델을 RetrievalQA 객체에 포함시켜 활용 가능
from langchain.chains import RetrievalQA

# ChatOpenAI : LangChain에서 지원하는 OpenAI 대화 전용 모델
# 기존의 completion 객체는 입력 형태가 맞지 않아 RetrievalQA에 직접 활용할 수 없음
from langchain.chat_models import ChatOpenAI
chat_model = ChatOpenAI(
    model_name = "gpt-3.5-turbo",
    openai_api_key = MY_API_KEY
)

C:\Dev\tools\Anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
C:\Dev\tools\Anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [21]:
qa1 = RetrievalQA.from_chain_type(
    llm = chat_model,
    # chain_type = 답변을 위해 청크들이 결합되는 타입을 지정
    # 1. stuff : 검색된 청크들의 내용을 간단히 결합하여 응답 생성
    # 2. map_reduce : 청크들을 각각 처리한 후 개별 결과를 결합하여 응답 생성 (큰 규모 데이터)
    # 3. refine : 이전 청크의 처리 결과를 다음 청크에 반영하고 지속적으로 개선하여 응답 생성
    chain_type = "refine",
    retriever = my_retriever,
    return_source_documents = True # 어떤 내용을 참조해서 응답을 만들었는지 출력
)

In [22]:
question = "기술 평가가 뭐야?"
llm_res = qa1(question)
llm_res

C:\Dev\tools\Anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'query': '기술 평가가 뭐야?',
 'result': '기술평가는 대상 기술의 기술성, 시장성, 사업타당성 등을 분석하여 기술가치를 산정하고, 이를 담보가치나 소송가액 등 다양한 용도로 활용하는 과정을 말합니다. 또한, 기술평가를 통해 기업이 보유한 콘텐츠나 주식에 대한 가치평가를 실시하여 가액을 산정하는 등 다양한 기술 관련 평가를 포함합니다. 기술이전이나 합병 및 인수(M&A) 등의 기술거래 시에는 당사자의 의뢰에 의해 기술평가가 실시되며, 기술의 객관적인 가치를 산정하게 됩니다. 이러한 기술평가는 기업이 상장하거나 기술거래를 위한 기술이전을 진행할 때 중요한 역할을 합니다.',
 'source_documents': [Document(page_content='페이지  2 / 6 \n \n (2) 기술평가3 \n기술평가란 ?- 대상기술의  기술성 , 시장성 , 사업타당성  등을 분석하고  결과를  금액, 등급, 의견 등으로  \n표현하는  것이다 .  \n✓ 기술평가는  기술수준  등 기술자체에  대한 평가를  표현한  용어지만 , 기술과  기업(사업)간의 밀접한  관련\n성으로  인해서  최근에는  위의 의미로  사용됨  \n✓ 기술가치평가는  기술평가 +가치평가가  결합된  용어로서 , 기술적인  요소를  기반으로  시장에서의  가치를  \n평가 하는 것을 의미 \n \n (3) 기술평가  유형 \n 정 의 세부평가종류  \n기술가치평\n가 대상 기술에  의하여  현재 시현되고  있거나  \n장래에  시현될  기술의  가치를  평가하여  평\n가결과를  금액으로  표시 - 벤처기업  현물출자  특례대상  산업재산권   등의 평가 \n- 기술의  담보가치를  산정하기  위한 평가 \n- 기술이전ㆍ거래4 기준가격  산정을  위한   평가 \n- 기술사업의  이전ㆍ양수도를  위한 평가 등  \n기술사업  \n타당성평가  기업이  특정기술 (OR 아이디어)을 사업화 , \n추진중인  기술사업의  투자 확대 시 사업의  \n기술성  및 사업타당성  평가 

### Reranking 사용
- Reranking을 지원하는 cohere 라이브러리 사용 (cohere사에서 지원)
- https://dashboard.cohere.com/api-keys 에서 Key 발급 필요 (가입 시, 디폴트로 발급)
- Reranking을 적용하며 검색된 내용 중에서 핵심 문장만을 요약 추출하는 다른 검색기도 활용

In [23]:
# CohereRerank : LangChain에서 지원하는 Reranking 구현 클래스
from langchain.retrievers.document_compressors import CohereRerank

# ContextualCompressionRetriever : 검색 된 문서의 문맥을 압축하여 보다 집중적이고 관련성 높은 정보를 제공하는 검색기 (Rerank 객체와 결합 가능)
# 문맥 압축은 가장 관련성이 높은 부분을 선택하고 덜 중요한 정보는 삭제하여 핵심만 남겨두는 것
from langchain.retrievers import ContextualCompressionRetriever

In [24]:
COHERE_API_KEY = getpass("Cohere API KEY >> ")

In [25]:
# CohereRerank 객체 생성
my_rerank = CohereRerank(
    cohere_api_key = COHERE_API_KEY,
)

# 검색시 객체 설정
compression_retriever = ContextualCompressionRetriever(
    base_compressor = my_rerank,
    base_retriever = my_retriever
)

C:\Dev\tools\Anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `CohereRerank` was deprecated in LangChain 0.0.30 and will be removed in 0.3.0. An updated version of the class exists in the langchain-cohere package and should be used instead. To use it run `pip install -U langchain-cohere` and import as `from langchain_cohere import CohereRerank`.
  warn_deprecated(


In [26]:
qa2 = RetrievalQA.from_chain_type(
    llm = chat_model,
    chain_type = "refine",
    retriever = compression_retriever # rerank 기능 포함
)

- %%time : 해당 셀 실행 시간 측정 (주피터노트북에서 지원하는 기능)

In [27]:
%%time
question = "기술 평가가 뭐야?"
print(qa2.run(question))

C:\Dev\tools\Anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


기술 평가는 대상 기술의 기술성, 시장성, 사업타당성 등을 분석하고 결과를 금액, 등급, 의견 등으로 표현하는 것을 말합니다. 이를 통해 기술의 가치나 잠재력을 평가하고 이를 기반으로 다양한 결정이나 투자 등을 할 수 있습니다. 또한, 기술이전 및 M&A 거래시 당사자의 의뢰에 의해 가치평가를 실시하고, 기술의 객관적인 가치를 산정하여 거래를 원활하게 할 수 있습니다. 이를 통해 기술이전이나 M&A 거래 과정에서의 중요한 역할을 수행할 수 있습니다.
CPU times: total: 266 ms
Wall time: 9.81 s


- RAG 구조와 벡터 DB를 활용했을 때, 소량의 데이터를 저장했음에도 일반적인 질의에 비해 훨씬 구체적이고 세부적인 답변이 나오는 것을 확인할 수 있으며, 모델을 재학습 시키는 Fine-tuning에 비해 간단하고, 기업이나 개인의 데이터가 유출될 가능성을 낮출 수 있음